Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using StatisticalRethinking
using DynamicHMC, TransformVariables, LogDensityProblems, MCMCDiagnostics
using Parameters, ForwardDiff, LinearAlgebra

┌ Info: Recompiling stale cache file /Users/rob/.julia/compiled/v1.2/DynamicHMC/iPqvI.ji for DynamicHMC [bbc10e6e-7c05-544b-b16e-64fede858acb]
└ @ Base loading.jl:1187
┌ Info: Recompiling stale cache file /Users/rob/.julia/compiled/v1.2/MCMCDiagnostics/kMBOZ.ji for MCMCDiagnostics [6e857e4b-079a-58c4-aeab-bc2670384359]
└ @ Base loading.jl:1187


### snippet 10.4

In [2]:
d = CSV.read(rel_path("..", "data", "chimpanzees.csv"), delim=';');
df = convert(DataFrame, d);
df[:pulled_left] = convert(Array{Int64}, df[:pulled_left])
df[:prosoc_left] = convert(Array{Int64}, df[:prosoc_left])
first(df, 5)

struct m_10_02d_model{TY <: AbstractVector, TX <: AbstractMatrix}
    "Observations."
    y::TY
    "Covariates"
    X::TX
end

Make the type callable with the parameters *as a single argument*.

In [3]:
function (problem::m_10_02d_model)(θ)
    @unpack y, X, = problem   # extract the data
    @unpack β = θ  # works on the named tuple too
    ll = 0.0
    ll += logpdf(Normal(0, 10), β[1]) # a = X[1]
    ll += logpdf(Normal(0, 10), β[2]) # bp = X[2]
    ll += sum([loglikelihood(Binomial(1, logistic(dot(X[i, :], β))), [y[i]]) for i in 1:N])
    ll
end

Instantiate the model with data and inits.

In [4]:
N = size(df, 1)
X = hcat(ones(Int64, N), df[:prosoc_left]);
y = df[:pulled_left]
p = m_10_02d_model(y, X);
θ = (β = [1.0, 2.0],)
p(θ)

-487.6540051035728

Write a function to return properly dimensioned transformation.

In [5]:
problem_transformation(p::m_10_02d_model) =
    as( (β = as(Array, size(p.X, 2)), ) )
# Wrap the problem with a transformation, then use Flux for the gradient.
P = TransformedLogDensity(problem_transformation(p), p)
∇P = ADgradient(:ForwardDiff, P);

Tune and sample.

In [6]:
chain, NUTS_tuned = NUTS_init_tune_mcmc(∇P, 1000);

MCMC, adapting ϵ (75 steps)
0.0025 s/step ...done
MCMC, adapting ϵ (25 steps)
0.0011 s/step ...done
MCMC, adapting ϵ (50 steps)
0.0035 s/step ...done
MCMC, adapting ϵ (100 steps)
0.00071 s/step ...done
MCMC, adapting ϵ (200 steps)
0.00054 s/step ...done
MCMC, adapting ϵ (400 steps)
0.00055 s/step ...done
MCMC, adapting ϵ (50 steps)
0.00066 s/step ...done
MCMC (1000 steps)
0.00041 s/step ...done


We use the transformation to obtain the posterior from the chain.

In [7]:
posterior = TransformVariables.transform.(Ref(∇P.transformation), get_position.(chain));
posterior[1:5]

5-element Array{NamedTuple{(:β,),Tuple{Array{Float64,1}}},1}:
 (β = [0.218199, 0.350468],)  
 (β = [-0.0849741, 0.786689],)
 (β = [-0.180192, 0.780655],) 
 (β = [-0.135587, 0.687399],) 
 (β = [0.146955, 0.575457],)  

Extract the parameter posterior means: `β`,

In [8]:
posterior_β = mean(first, posterior)

2-element Array{Float64,1}:
 0.04556270637112988
 0.5614209317848208 

Effective sample sizes (of untransformed draws)

In [9]:
ess = mapslices(effective_sample_size, get_position_matrix(chain); dims = 1)
ess

1×2 Array{Float64,2}:
 885.605  950.894

NUTS-specific statistics

In [10]:
NUTS_statistics(chain)

Hamiltonian Monte Carlo sample of length 1000
  acceptance rate mean: 0.88, min/25%/median/75%/max: 0.06 0.81 0.94 1.0 1.0
  termination: AdjacentTurn => 13% DoubledTurn => 87%
  depth: 1 => 27% 2 => 73%


CmdStan result

In [11]:
m_10_2s_result = "
Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
      Mean        SD       Naive SE       MCSE      ESS
 a 0.05103234 0.12579086 0.0019889282 0.0035186307 1000
bp 0.55711212 0.18074275 0.0028577937 0.0040160451 1000

Quantiles:
       2.5%        25.0%       50.0%      75.0%      97.5%
 a -0.19755400 -0.029431425 0.05024655 0.12978825 0.30087758
bp  0.20803447  0.433720250 0.55340400 0.67960975 0.91466915
";

Extract the parameter posterior means: `β`,

In [12]:
posterior_β = mean(first, posterior)

2-element Array{Float64,1}:
 0.04556270637112988
 0.5614209317848208 

End of `10/m10.02d.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*